In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ethereum-phishing-transaction-network/Ethereum Phishing Transaction Network/README.txt
/kaggle/input/ethereum-phishing-transaction-network/Ethereum Phishing Transaction Network/MulDiGraph.pkl


In [38]:
import platform
import pickle
import networkx as nx

In [39]:
def load_pickle(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

In [40]:
import random
def getRandom():
    random.seed(2973489)
    random_list=[]
    for i in range(10):
        random_list.append(random.randint(1,2973489))
    return random_list

In [41]:
def getPhishingSet(G):
    phishingSet=set()
    for idx,nd in enumerate(nx.nodes(G)):
        if G.nodes[nd]['isp']==1:
            phishingSet.add(nd)
    return phishingSet

In [42]:
import random

def RandomSelectNode(G,randomNum,size):

    legal_node_set=set()
    for ind,nd in enumerate(nx.nodes(G)):

        if ind>=randomNum:
            
            legal_node_set.add(nd)
            suc=list(G.successors(nd))
            pre=list(G.predecessors(nd))

            for i in range(len(suc)):
                if len(legal_node_set)>=size:
                    break
                legal_node_set.add(suc[i])

            for i in range(len(pre)):
                if len(legal_node_set)>=size:
                    break
                legal_node_set.add(pre[i])


            while len(legal_node_set)<size:
                isWho=random.randint(0,1)
                isOut=False
                node_select=None
                if len(suc)==0:
                    isWho=1
                if len(pre)==0:
                    isWho=0
                if isWho==0:
                    choose=random.randint(0,len(suc)-1)
                    node_select=suc[choose]
                if isWho==1:
                    choose=random.randint(0,len(pre)-1)
                    node_select=pre[choose]
                legal_node_set.add(node_select)
                suc=list(G.successors(node_select))
                pre=list(G.predecessors(node_select))
                for i in range(len(suc)):
                    if len(legal_node_set)>=size:
                        isOut=True
                        break
                    legal_node_set.add(suc[i])

                if isOut:
                    break
                for i in range(len(pre)):
                    if len(legal_node_set)>=size:
                        isOut=True
                        break
                    legal_node_set.add(pre[i])

                if isOut:
                    break
            if len(legal_node_set)>=size:
                break
    return legal_node_set

In [43]:
import csv

def RandomLabel(legal_node_set,G):

    legal_node_list=list(legal_node_set)
    with open('RandomLabel1.csv','w',newline='') as file:
        writer=csv.writer(file)
        writer.writerow(['account','isp'])
        for i in range(len(legal_node_list)):
            data=[]
            data.append(legal_node_list[i])
            data.append(G.nodes[legal_node_list[i]]['isp'])
            writer.writerow(data)

In [44]:
def getNodeData(legal_node_list,G):
    node_data=[]
    for i in range(len(legal_node_list)):
        node=legal_node_list[i]
        node_data_temp=[]
        time=[]
        time_min=99999999
        time_max=0
        amount=[]
        in_time=[]
        in_time_min=99999999
        in_time_max=0
        in_amount=[]
        node_set=set()
        pre=list(G.predecessors(node))
        for j in range(len(pre)):
            for v in G[pre[j]][node].values():
                in_time.append(v['amount'])
                in_amount.append(v['timestamp'])
                if v['timestamp']>in_time_max:
                    in_time_max=v['timestamp']
                if v['timestamp']<in_time_min:
                    in_time_min=v['timestamp']


        for i,v in dict(G[node]).items():
            node_set.add(i)
            for v1 in v.values():
                time.append(v1['timestamp'])
                amount.append(v1['amount'])
                if v1['timestamp']>time_max:
                    time_max=v1['timestamp']
                if v1['timestamp']<time_min:
                    time_min=v1['timestamp']

        if len(time)>0:
            out_block_std=statistics.pstdev(time)
            out_time_ptp=time_max-time_min
            amount_sum=sum(amount)
            amount_max=max(amount)
            amount_min=min(amount)
            amount_mean=statistics.mean(amount)
            amount_std=statistics.pstdev(amount)
            count=len(time)
            unique=len(node_set)
            unique_ratio=unique/count

        if len(time)<=0 :
            out_block_std=0
            out_time_ptp=0
            amount_sum=0
            amount_max=0
            amount_min=0
            amount_mean=0
            amount_std=0
            count=0
            unique=0
            unique_ratio=0

        if len(in_time)>0:
            in_block_std=statistics.pstdev(in_time)
            in_time_ptp=in_time_max-in_time_min
            in_amount_sum=sum(in_amount)
            in_amount_max=max(in_amount)
            in_amount_min=min(in_amount)
            in_amount_mean=statistics.mean(in_amount)
            in_amount_std=statistics.pstdev(in_amount)
            in_count=len(in_time)
            in_unique=len(node_set)
            in_unique_ratio=in_unique/in_count

        if len(in_time)<=0:
            in_block_std=0
            in_time_ptp=0
            in_amount_sum=0
            in_amount_max=0
            in_amount_min=0
            in_amount_mean=0
            in_amount_std=0
            in_count=0
            in_unique=0
            in_unique_ratio=0

        node_data_temp.append(node)
        node_data_temp.append(out_block_std)
        node_data_temp.append(out_time_ptp)
        node_data_temp.append(amount_sum)
        node_data_temp.append(amount_max)
        node_data_temp.append(amount_min)
        node_data_temp.append(amount_mean)
        node_data_temp.append(amount_std)
        node_data_temp.append(count)
        node_data_temp.append(unique)
        node_data_temp.append(unique_ratio)

        node_data_temp.append(in_block_std)
        node_data_temp.append(in_time_ptp)
        node_data_temp.append(in_amount_sum)
        node_data_temp.append(in_amount_max)
        node_data_temp.append(in_amount_min)
        node_data_temp.append(in_amount_mean)
        node_data_temp.append(in_amount_std)
        node_data_temp.append(in_count)
        node_data_temp.append(in_unique)
        node_data_temp.append(in_unique_ratio)





        node_data.append(node_data_temp)
    return node_data

In [45]:
def getAllToFriendsOutDirectionFeatures(legal_node_list,G):
    To_Out_Fea=[]
    for i in range(len(legal_node_list)):
        To_Out_Fea_Temp=[]
        To_Out_Fea_Temp.append(legal_node_list[i])
        suc=list(G.successors(legal_node_list[i]))
        to_out_sum_media=0
        to_out_min_std=0
        to_out_sum_min=0
        media=[]
        std=[]
        min_list=[]
        for j in range(len(suc)):
            cal_amount=[]

            temp_min=999999999
            for i,v in dict(G[suc[j]]).items():
                for v1 in v.values():
                    cal_amount.append(v1['amount'])
                    if temp_min>v1['amount']:
                        temp_min=v1['amount']
            if len(cal_amount)>0:
                media.append(statistics.mean(cal_amount))
                std.append(statistics.pstdev(cal_amount))
                min_list.append(temp_min)
        if len(media)>0:
            to_out_sum_media=sum(media)
        if len(media)<=0:
            to_out_sum_media=0
        if len(std)>0:
            to_out_min_std=min(std)
        if len(std)<=0:
            to_out_min_std=0
        if len(min_list)>0:
            to_out_sum_min=sum(min_list)
        if len(min_list)<=0:
            to_out_sum_min=0
        To_Out_Fea_Temp.append(to_out_sum_media)
        To_Out_Fea_Temp.append(to_out_min_std)
        To_Out_Fea_Temp.append(to_out_sum_min)

        To_Out_Fea.append(To_Out_Fea_Temp)
    return  To_Out_Fea


In [46]:
import statistics
def getAllFromFriendsInDirectionFeatures(legal_node_list,G):
    From_In_Fea=[]
    for i in range(len(legal_node_list)):
        From_In_Fea_Temp=[]
        From_In_Fea_Temp.append(legal_node_list[i])
        pre=list(G.predecessors(legal_node_list[i]))
        from_in_sum_min=0
        from_in_min_std=0
        from_in_min_sum=0
        from_in_mean_sum=0
        sum_list=[]
        std=[]
        min_list=[]
        for j in range(len(pre)):
            cal_amount=[]
            temp_min=9999999
            preFriend=list(G.predecessors(pre[j]))
            for k in range(len(preFriend)):
                for v in G[preFriend[k]][pre[j]].values():
                    cal_amount.append(v['amount'])
                    if temp_min>v['amount']:
                        temp_min=v['amount']
            if len(cal_amount)>0:
                sum_list.append(sum(cal_amount))
                std.append(statistics.pstdev(cal_amount))
                min_list.append(temp_min)
        if len(min_list)>0:
            from_in_sum_min=sum(min_list)
        if len(min_list)<=0:
            from_in_sum_min=0
        if len(std)>0:
            from_in_min_std=min(std)
        if len(std)<=0:
            from_in_min_std=0
        if len(sum_list)>0:
            from_in_min_sum=min(sum_list)
            from_in_mean_sum=statistics.mean(sum_list)
        if len(sum_list)<=0:
            from_in_min_sum=0
            from_in_mean_sum=0
        From_In_Fea_Temp.append(from_in_sum_min)
        From_In_Fea_Temp.append(from_in_min_std)
        From_In_Fea_Temp.append(from_in_min_sum)
        From_In_Fea_Temp.append(from_in_mean_sum)
        From_In_Fea.append(From_In_Fea_Temp)

    return From_In_Fea


In [47]:
import numpy as np
def getRandomFeatures(node_data,To_Out_Fea,From_In_Fea):
    node_data_np=np.array(node_data)
    To_Out_Fea_np=np.array(To_Out_Fea)
    To_Out_Fea_np=To_Out_Fea_np[:,1:]
    From_In_Fea_np=np.array(From_In_Fea)
    From_In_Fea_np=From_In_Fea_np[:,1:]
    node_features=np.hstack((node_data_np,To_Out_Fea_np))
    node_features=np.hstack((node_features,From_In_Fea_np))
    
    with open('RandomNodeFeatures.csv','w',newline='') as file:
        writer=csv.writer(file)
        writer.writerow(['account','out_block_std','out_time_ptp','out_amount_sum','out_amount_max','out_amount_min'
                    ,'out_amount_mean','out_amount_std','out_count','out_unique',
                    'out_unique_ratio',
                    'in_block_std','in_time_ptp','in_amount_sum','in_amount_max','in_amount_min'
                    ,'in_amount_mean','in_amount_std','in_count','in_unique',
                    'in_unique_ratio','to_out_sum_media','to_out_min_std','to_out_sum_min',
                     'from_in_sum_min','from_in_min_std','from_in_min_sum','from_in_mean_sum'
                     
                    ])
        for i in range(len(node_features)):
            writer.writerow(node_features[i])

In [48]:
G = load_pickle('../input/ethereum-phishing-transaction-network/Ethereum Phishing Transaction Network/MulDiGraph.pkl')
random_list=getRandom()
phishingSet=getPhishingSet(G)
print(random_list[0])
legal_node_set=RandomSelectNode(G,random_list[0],2600)
count=0
for node in iter(phishingSet):
    if node not in legal_node_set:
        legal_node_set.add(node)
        count+=1
    if count>=100:
        break    
legal_node_list=list(legal_node_set)
node_data=getNodeData(legal_node_list,G)
To_Out_Fea=getAllToFriendsOutDirectionFeatures(legal_node_list,G)
From_In_Fea=getAllFromFriendsInDirectionFeatures(legal_node_list,G)
RandomLabel(legal_node_set,G)
getRandomFeatures(node_data,To_Out_Fea,From_In_Fea)

1921733


In [49]:
for i in range(len(node_data)):
    if (node_data[i][0]!=To_Out_Fea[i][0]) or (node_data[i][0]!=From_In_Fea[i][0]) or (To_Out_Fea[i][0]!=From_In_Fea[i][0]):
        print("数据没对上")
        break
print("顺利结束")

顺利结束


In [50]:
RandomLabel(legal_node_set,G)
getRandomFeatures(node_data,To_Out_Fea,From_In_Fea)